## Set paths

In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import timeit
#import pylab 

#import re
#import seaborn as sns

##### SET PARAMETERS ##########################

# path of bridge data for reading csv file (_roads2)
path     = 'D:\Jupyter_File\Advanced_Discrete_Cleaning\WBSIM_Lab2\infrastructure\_roads3.csv'
# path of bridge data for reading Excel file (BMMS_overview)
bpath     = 'D:\Jupyter_File\Advanced_Discrete_Cleaning\WBSIM_Lab2\infrastructure\BMMS_overview.xlsx'

# path for saving excel for simulation
#savepath  = 'C:\\Users\\Marti\\Documents\\EPA1351_Advanced_Discrete_Simulation\\Assignment\\Assignment 2\\WBSIM_Lab2\\infrastructure\\simio_test.xlsx'


n         = 628 # number of bridges, maximum = 628
spd       = 13.33 # speed of truck [km/hr]

## import data

In [2]:
# reading raw data
df_orig  = pd.read_csv(path,index_col=None, header=0)
bdf_orig = pd.read_excel(bpath, index_col=None, header=0)

# copy dataframe
df  = df_orig.copy()
bdf = bdf_orig.copy()

## select N1 data

In [3]:
# select based on N1 name
df  = df[(df['road'] == 'N1') & (df['lat'] > 22.3677155) & (df['lon'] < 91.7852225)]
bdf = bdf[(bdf['road'] == 'N1') & (bdf['lat'] > 22.3677155) & (bdf['lon'] < 91.7852225)]

In [4]:
# drop out the duplicates of bridges
print('bridge data before duplicate drop: ', len(bdf))
bdf = bdf.drop_duplicates(subset=['chainage'], keep='first')
print('bridge data after diplicate drop: ', len(bdf))

bridge data before duplicate drop:  271
bridge data after diplicate drop:  210


## Create entity sheetframe

In [5]:
# Add columns to bridge data set
bdf['Object Class']               = 'Bridge'
bdf['Y']                          = 0
bdf['RunInitializedAddOnProcess'] = 'BreakBridgeOnProb'
bdf['EnteredAddOnProcess']        = 'SetDelayTime'
bdf['Height']                     = np.nan

# rename wrong columns
bdf.rename(columns={'condition' : 'Quality','length': 'Length', 'LRPName': 'Object Name','lat':'X', 'lon':'Z',
                   'width':'Width'}, inplace = True)

# drop unnecessary columns
bdf = bdf.drop(['road','km','type','name','structureNr','roadName','EstimatedLoc','constructionYear',
                'zone','circle','spans','division','sub-division'],axis=1)


In [6]:
# select part of dataframe
objects = bdf.iloc[0:n,:]

# adding source and sink

# set model entity
model = pd.DataFrame([['ModelEntity','Truck',spd,23,0,90]],
                     columns=['Object Class', 'Object Name', 'InitialDesiredSpeed','Z','Y','X'])

# set source
source = pd.DataFrame([[ 'Chittagong',df.loc[0,'chainage'],df.loc[0,'lat'],0,df.loc[0,'lon'],'Source','truck',5]],
                      columns=['Object Name', 'chainage', 'Z', 'Y','X','Object Class', 'EntityType','InterarrivalTime'])


# set sink
sink   = pd.DataFrame([[ 'Dhaka',df.loc[len(df)-1,'chainage'],df.loc[len(df)-1,'lat'],0,df.loc[len(df)-1,'lon'],'Sink']],
                     columns=['Object Name','chainage','Z','Y','X','Object Class'])


# add source and sink to dataframe
objects = pd.concat([model, source, objects, sink], axis=0)

In [7]:
# Define midpoint of road
midX = 90
midZ = 23

# change and scale X and Z coordinates of road
objects['X'] = (objects['X']-midX)*101
objects['Z'] = (objects['Z']-midZ)*-111

In [8]:
# sort values 
objects = objects.sort_values(by='chainage')

# reindex dataframe
objects = objects.reset_index(drop=True)

In [9]:
#print(objects)

## create links sheet

In [10]:
# create empty dataframe
links = pd.DataFrame([],columns=['Link Class','Link Name','From Node','To Node','Network','Type','LogicalLength','DrawnToScale'])

# set values in frame
links['LogicalLength'] = (abs(objects['chainage'].diff())*1000 ).values
links['Link Class']    = 'Path'
links['DrawnToScale']  = 'False'
links['Type']          = 'Unidirectional'
links['Network']       = 'Network A'



# create paths
for i in range(len(objects)-1):
    
    #print(objects[i]['Name'])
    links['Link Name'].iloc[i+1] = (objects.loc[i,'Object Name'] + '_' + objects.loc[i+1,'Object Name'])
    links['From Node'].iloc[i+1] = (objects.loc[i,'Object Name'])
    links['To Node'].iloc[i+1]   = (objects.loc[i+1,'Object Name'])
   

# extra formatting
links.loc[2,'From Node']          = 'Output@Chittagong' 
links.loc[len(links)-1,'To Node'] = 'Input@Dhaka'
links                             = links.drop(links.index[[0,1]]) 

D:\Anaconda\lib\site-packages\pandas\core\indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [11]:
#print(links)

In [12]:
# rearrange columns of dataframes
objects = objects[['Object Class','Object Name','X','Y','Z','Length','Width','Height','InterarrivalTime','InitialDesiredSpeed'
                   ,'EntityType','RunInitializedAddOnProcess','EnteredAddOnProcess','Quality','Length']]


In [ ]:
plt.figure(figsize = (8,10))
plt.scatter(x= dfN1['lon'], y= dfN1['lat'],marker='o')
plt.scatter(x= bdfN1['lon'], y= bdfN1['lat'],marker='+')

## convert to excel

In [13]:
# open writer object
writer = pd.ExcelWriter('savepath.xlsx', engine='xlsxwriter')

# write sheets
objects.to_excel(writer, sheet_name='Objects1', index=False)
links.to_excel(writer, sheet_name='Links1', index=False)

#Close the Pandas Excel writer and output the Excel file.
writer.save()